# Machine Learning Decal, Spring 2018
## Day 3: Linear Regression

In [ ]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

### NBA Salary Data

In [ ]:
nba_sals = pd.read_csv("./nbasalary.csv", index_col = 0)
nba_sals = nba_sals.dropna(axis=0)
nba_sals.head()

In [ ]:
log_wage = nba_sals["lwage"]
wage = nba_sals["wage"]
points = nba_sals["points"]
exper = nba_sals["exper"]

### Simple Linear Regression

##### In this section, we will compare two SLR models and see which one performs better on a validation set. 
##### Model 1: Regressing wage on points scored
##### Model 2: Regressing wage on years of experience

In [ ]:
plt.figure()
plt.title("Wage vs. Points")
plt.xlabel("Points")
plt.ylabel("Wage")
######################
## CODE HERE
######################

plt.figure()
plt.title("Wage vs. Experience")
plt.xlabel("Experience")
plt.ylabel("Wage")
######################
## CODE HERE
######################

plt.show()

#### A little validation...

In [ ]:
wage_train = nba_sals["wage"][0:214]
wage_valid = nba_sals["wage"][214:]
points_train = nba_sals["points"][0:214]
points_valid = nba_sals["points"][214:]
exper_train = nba_sals["exper"][0:214]
exper_valid = nba_sals["exper"][214:]

#### Regression wage on points...

In [ ]:
######################
## CODE HERE
######################
print("The MSE for the model wage~points is:", mse)

#### Regressing wage on experience...

In [ ]:
######################
## CODE HERE
######################
print("The MSE for the model wage~experience is:", mse)

##### Conclusion: points scored is a better predictor of wage than years of experience

### Multiple Linear Regression

#### Wage vs. Experience & Points

In [ ]:
exp_dat = nba_sals[["exper","points"]]
exp_dat = sm.add_constant(exp_dat)
myMLR = sm.OLS(wage, exp_dat).fit()
myMLR.summary()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x1, x2 = np.meshgrid(np.linspace(exp_dat.exper.min(), exp_dat.exper.max(), 100), 
                       np.linspace(exp_dat.points.min(), exp_dat.points.max(), 100))

x3 = myMLR.params[0] + myMLR.params[1] * x1 + myMLR.params[2] * x2

# create matplotlib 3d axes
fig = plt.figure(figsize=(12, 8))
my3D = Axes3D(fig, azim=-120, elev=20)

# plot hyperplane
surf = my3D.plot_surface(x1, x2, x3, cmap=plt.cm.RdBu_r, alpha=0.5, linewidth=0.5)

# plot data points
resid = wage - myMLR.predict(exp_dat)
my3D.scatter(exp_dat[resid >= 0].exper, exp_dat[resid >= 0].points, wage[resid >= 0], color='black', alpha=1.0, facecolor='white')
my3D.scatter(exp_dat[resid < 0].exper, exp_dat[resid < 0].points, wage[resid < 0], color='black', alpha=1.0)

# set axis labels
my3D.set_xlabel('experience')
my3D.set_ylabel('points')
my3D.set_zlabel('wage')
my3D.set_title('Regression Plane in 3D')

plt.show()

### Polynomial Regression

We have our data with 2 independent variables - experience and points. Let's call them $\alpha_1$ and $\alpha_2$ for now.

So far, we have just been using the indep variables as features. So, our model was something like $$ h_{\theta}(\alpha) = \theta_0 + \theta_1*\alpha_1 + \theta_2*\alpha_2 $$

But what if we didn't want to have our features be just the variables? What if we wanted the features to be the square of the variables, or something more fancy? Such as this:

$$ h_{\theta}(\alpha) = \theta_0 + \theta_1*\alpha_1^2 + \theta_2*\alpha_2*\alpha_1 $$

In fact, this can very easily done with polynomial features. Every polynomial has a degree. The degree controls to what exponent the combined terms are. In fact, what we have been doing so far is creating polynomial features with degree d=1! Let's see what happens when d=2:

$$ h_{\theta}(\alpha) = \theta_0 + \theta_1*\alpha_1 + \theta_2*\alpha_2 + \theta_3*\alpha_1^2 + \theta_4*\alpha_2^2 + \theta_5*\alpha_1*\alpha_2 $$

What we see is we add more _cross terms_ of our features to create higher-order representations. Notice our model is still linear, though, since each $\theta$ is still only first degree (linear).

In [ ]:
def poly_reg(x, y, degree = 1):
    # Add a bias term to the dataset
    ######################
    ## CODE HERE
    ######################
    
    # Create polynomial features
    ######################
    ## CODE HERE
    ######################

    # Split into training and validation set
    split = int(0.8*x.shape[0])
    x_train, x_val = x[0:split, ], x[split:, ]
    y_train, y_val = y[:split, ], y[split:, ]
    
    # Fit the polynomial regression model
    my_reg = sm.OLS(y_train, x_train).fit()
    
    # Make predictions
    val_preds = my_reg.predict(x_val)
    train_preds = my_reg.predict(x_train)
    val_mse = mean_squared_error(y_val, val_preds)
    train_mse = mean_squared_error(y_train, train_preds)
    print("Degree:", degree, "\n", 
          "Train MSE:", train_mse, "\n", "Valid MSE:", val_mse) 
    
    return train_mse, val_mse

In [ ]:
deg_list = [1, 2, 3, 4, 5]
t_mse, v_mse = [], []
for deg in deg_list:
    t, v = poly_reg(exp_dat, wage, deg)
    t_mse.append(t)
    v_mse.append(v)

In [ ]:
plot1 = plt.plot(deg_list, t_mse, '-ob', label = 'train')
plot2 = plt.plot(deg_list, v_mse, '-or', label = 'valid')
plt.ylabel("MSE")
plt.xlabel("Degree of polynomial features")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()